In [2]:
import json
from elasticsearch import Elasticsearch, helpers
import datetime
from glob import glob
es = Elasticsearch()

def parse_date(tweet):
    tweet['created_at'] = datetime.datetime.strptime(tweet['created_at'],'%a %b %d %H:%M:%S +0000 %Y')
    tweet['user']['created_at'] = datetime.datetime.strptime(tweet['user']['created_at'],'%a %b %d %H:%M:%S +0000 %Y')
    if ('retweeted_status' in tweet):
        tweet['retweeted_status']['created_at'] = datetime.datetime.strptime(tweet['retweeted_status']['created_at'],'%a %b %d %H:%M:%S +0000 %Y')
        tweet['retweeted_status']['user']['created_at'] = datetime.datetime.strptime(tweet['retweeted_status']['user']['created_at'],'%a %b %d %H:%M:%S +0000 %Y')
    return tweet

In [ ]:
forest500settings = {
    "settings":{
        "index":{
            "mapping":{
                "total_fields":{
                    "limit":"2000"
                }
            }
        }
    }
}
es.indices.create('forest500',body=forest500settings)

In [ ]:
#companies_path = glob('/Users/luiz/OneDrive/forest500/output-company/*.json')
companies_path = glob('../data/company/*.json')
for company_path in companies_path:
    with open(company_path) as f:
        tweets = json.load(f)
    
    print(company_path)

    for tweet in tweets:
        tweet = parse_date(tweet)
        tweet['user']['type'] = 'company'
        res = es.index(index="forest500", id=tweet['id'], body=tweet)

es.indices.refresh(index="forest500")

In [ ]:
#companies_path = glob('/Users/luiz/OneDrive/forest500/output/*.json')
companies_path = glob('../data/if/*.json')
for company_path in companies_path:
    with open(company_path) as f:
        tweets = json.load(f)
    
    print(company_path)

    for tweet in tweets:
        tweet = parse_date(tweet)
        tweet['user']['type'] = 'financial'
        res = es.index(index="forest500", id=tweet['id'], body=tweet)

es.indices.refresh(index="forest500")

In [ ]:
query = {
  "query": {
    "query_string": {
      "query": "SantanderAM_UK",
      "default_field": "user.screen_name"
    }
  }
}

res = es.search(index="forest500", body=query)


In [ ]:
import csv

signatories = {}
companies = {}
financial = {}

with open('/Users/luiz/OneDrive/forest500/input/list of signatories.csv') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';')
    for row in spamreader:
        signatories[row[1]] =  row[2:]

with open('/Users/luiz/Downloads/companies.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';', quotechar='"')
    for row in spamreader:
        if(len(row) > 1):
            companies[row[0]] = row[1]

with open('/Users/luiz/Downloads/financial.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';', quotechar='"')
    for row in spamreader:
        if(len(row) > 1):
            financial[row[0]] = row[1]


In [4]:
import pandas as pd

companies = pd.read_excel('../input/forest500.xlsx',sheet_name='company')
financial = pd.read_excel('../input/forest500.xlsx',sheet_name='financial')

for index,row in companies.iterrows():
    if(pd.isna(row['Handles'])):
        continue
    handles = row['Handles'].split(',')
    for handle in handles:
        print(handle)
        query = {
            "query": {
                "term": {
                    "user.screen_name.keyword": {
                        "value": handle,
                        "boost": 1.0
                    }
                }
            },
            "_source": False
        }


        res = helpers.scan(
                        client = es,
                        query = query, 
                        index = 'forest500')

        for tweet in res:
            update_query = {
                'doc':{
                    
                        'official_name': handle,
                        'type': row['Type'],
                        'subsidary': row['Subsidary'],
                        'brands': str(row['Brands']).split(','),
                        'sectors': str(row['Company sectors']).split(','),
                        'headquarters': row['Headquarters'],
                        'region': row['HQ region'],
                        'commodities_powerbroker': str(row['Commodities (powerbroker_']).split(','),
                        'segments_powerbroker': str(row['Segments (powerbroker)']).split(','),
                        'commodities_other': str(row['Commodities (Other)']).split(','),
                        'segments_other': str(row['Segments (Other)']).split(','),
                        'signatories': str(row['Sign']).split(','),
                }
            }
            try:
                es.update(index='forest500', id=tweet['_id'], body=update_query)
            except:
                print(update_query)


3M
agdagro
adidas
adidasfootball
adidasoriginals
adidasUK
adidasrunning
adidasUS
adidasHoops
adidasGolf
adidasalerts
adidassoccer
Adient
AdityaBirlaGrp
Advance_Digital
AEON_JAPAN
AFASCL
Agrifirm
AholdDelhaize
AldiUK
Allana_Sons
havaianaseurope
osklen
mizunousa 
amazon
AmazonUK
AMOREPACIFIC_US
Amul_Coop
Arauco_cl
ADMupdates
arcor
ArlaFoodsUK
AshleyHomeStore
ASICSamerica
Runkeeper
ASICSaustralia
 ASICSTennis
ASICSeurope
ASICS_ZA
ASICSUK
ASICSvolleyball
ASICS_India
ABFIngredients
adeo
BASFCorporation
BASFAgro
BASFAgProducts
BASF
BataCompanyNZ
beidahuanggroup
Bertelsmann_com
BESTGroupUK
BestGroupInt
BFG_Logistics
BioMarGroup
2SFGofficial
BRF_Brasil
bricapar
bright_food
bunge
clarksshoes
Calbee_JP
CampbellSoupCo
MichaelKors
 jimmychoo
 versace
cargill
CargillEMEA
Cargill_K12Food
CargillGrainUS
CargillAnimalUK
foodsecureworld
CarrefourSA
CarrefourGroup
CarrefourUAE
CarrefourUAE
CarrefourKe
Carrefour_qa
Groupe_Casino
CencosudArg
pr_CP
MengniuGroup
CSCECNEWS
clariant
ClariantOilSvcs
ClariantMB

In [5]:
import pandas as pd

financial = pd.read_excel('../input/forest500.xlsx',sheet_name='financial')

for index,row in financial.iterrows():
    if(pd.isna(row['Handles'])):
        continue
    handles = row['Handles'].split(',')
    for handle in handles:
        print(handle)
        query = {
            "query": {
                "term": {
                    "user.screen_name.keyword": {
                        "value": handle,
                        "boost": 1.0
                    }
                }
            },
            "_source": False
        }

        res = helpers.scan(
                        client = es,
                        query = query, 
                        index = 'forest500')

        for tweet in res:
            update_query = {
                'doc':{
                        'official_name': handle,
                        'type': row['Type'],
                        'subsidary': str(row['FI subsidary']),
                        'FI type': str(row['FI type']).split(','),
                        'headquarters': str(row['FI Headquarters']),
                        'region': str(row['HQ Region']),
                        'commodities': str(row['Commodities assessed for']).split(','),
                }
            }
            try:
                es.update(index='forest500', id=tweet['_id'], body=update_query)
            except:
                print(update_query)

ABNAMRO
Aegon
AMGFunds
Allianz
Allstate
AmericanCentury
AmericanEquity
amfam
AIGinsurance
ameriprise
AmBankMY
AQRCapital
ANZ_AU
AXA
AXA_Partners
AXANext
axainsurance
AXAIM
AXAVentures
BaillieGifford
bbva
BancodoBrasil
BankofAmerica
china_bank
BNYMellon
Barclays
blackrock
BlackRock_UK
 BLKPublicPolicy
BlackRockCA
BMO
bndes
bnpparibas
GroupeBPCE
bradesco
calpers
capitalgroup
charlesschwab
citi
commerzbank
commbank
rabobankgroup
creditagricole
cic
creditsuisse
dbsbank
deutschebank
dimensional
eatonvance
epfoaindia
farmcredit
farmermacnews
fidelity
fifththird
fti_global
gicsingapore 
goldmansachs
GSBSociety
guggenheimptnrs
homeloansbyhdfc
hsbc
icicibank
eximbank_id
ing_news
intesasanpaolo
InvescoCanada
invescoindia
InvescoUS
invescoemea
itau
janushenderson
jefferies
jpmorgan
kbank_live
kkr_co
kotakbankltd
Krungthai_Care
lbbw
lazard
landg_group
fti_global 
libertymutual
licindiaforever
lincolnfingroup
lbgplc
macquarie
Manulife
MassMutualUS
MetLife
MUFGAmericas
MizuhoAmericas
MorganStanley
N

In [ ]:
for company in companies:
    handles = companies[company].split(',')
    for handle in handles:
        handle = handle.replace(' ','').replace('/','').replace('\n','')
        print(handle)
        query = {
            "query": {
                "term": {
                    "user.screen_name": {
                        "value": handle,
                        "boost": 1.0
                    }
                }
            },
            "_source": False
        }

        res = helpers.scan(
                        client = es,
                        query = query, 
                        index = 'forest500')

        for tweet in res:
            update_query = {
                'doc':{
                    'user':{
                        'official_name': company
                    }
                }
            }
            es.update(index='forest500', id=tweet['_id'], body=update_query)
            


In [ ]:
for company in financial:
    if(company == '\ufeffFI Name'):
        continue
    handles = financial[company].split(',')
    for handle in handles:
        handle = handle.replace(' ','').replace('/','').replace('\n','')
        print(handle)
        query = {
            "query": {
                "term": {
                    "user.screen_name": {
                        "value": handle,
                        "boost": 1.0
                    }
                }
            },
            "_source": False
        }

        res = helpers.scan(
                        client = es,
                        query = query, 
                        index = 'forest500')

        for tweet in res:
            update_query = {
                'doc':{
                    'user':{
                        'official_name': company
                    }
                }
            }
            es.update(index='forest500', id=tweet['_id'], body=update_query)

In [ ]:
for signa in signatories:
    print(signa)
    names = set()

    query = {
      "query": {
        "term": {
          "user.official_name.keyword": {
            "value": signa,
            "case_insensitive": True
          }
        }
      },
        "fields": [
        "user.official_name.keyword"
      ],
        "_source": False
    }
    
    res = helpers.scan(
                    client = es,
                    query = query, 
                    index = 'forest500')

    for tweet in res:
      update_query = {
          'doc':{
              'user':{
                  'signatory': signatories[signa]
              }
          }
      }
      names.add(",".join(tweet['fields']['user.official_name.keyword']))
    es.update(index='forest500', id=tweet['_id'], body=update_query)
    if(not len(names)):
      print(names)


In [ ]:
import pandas as pd

df = pd.read_excel('../input/forest500.xlsx')

In [ ]:
df